In [1]:
import pandas as pd
import numpy as np
import math
import json
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
splitter=transcript.time >= 24*7*3
test_transcript=transcript[splitter].copy()
transcript=transcript[~splitter]

In [3]:
profile=profile.set_index('id')

portfolio=portfolio.set_index('id')
portfolio.index.rename('offer_id', inplace=True)

dummies=portfolio.channels.apply(lambda x: pd.Series(len(x)*[1], x)).fillna(0)
portfolio=pd.concat([portfolio.drop('channels', axis=1), dummies], axis=1)
portfolio.sort_values(['offer_type','reward','difficulty'])

,reward,difficulty,duration,offer_type,email,mobile,social,web
offer_id,,,,,,,,
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1.0,1.0,1.0,1.0
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,bogo,1.0,1.0,1.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,bogo,1.0,1.0,1.0,1.0
fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1.0,1.0,1.0,1.0
2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1.0,1.0,0.0,1.0
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,discount,1.0,1.0,1.0,1.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1.0,0.0,0.0,1.0
3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1.0,1.0,0.0,1.0


In [4]:
offers_made=transcript[transcript.event=='offer received'].copy()
offers_made['offer_id']=offers_made.value.apply(lambda x: x['offer id'])
offers_made.drop(['event', 'value'], axis=1, inplace=True)
offers_recieved=offers_made.merge(portfolio, left_on='offer_id', right_index=True)
offers_made=offers_made.groupby(['offer_id','time']).count().person
offers_made.name='count'


In [5]:
offers_made=offers_made.reset_index().merge(portfolio[['duration', 'reward']], left_on='offer_id', right_index=True)
offers_recieved=offers_recieved.merge(portfolio.duration, left_on='offer_id', right_index=True)

In [6]:
offers_made['end']=offers_made.time+offers_made.duration*24
offers_recieved['end']=offers_recieved.time+offers_made.duration*24

In [7]:
offers_recieved.head()

,person,time,offer_id,reward,difficulty,duration_x,offer_type,email,mobile,social,web,duration_y,end
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,240.0
18,ebe7ef46ea6f4963a7dd49f501b26779,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,120.0
21,f082d80f0aac47a99173ba8ef8fc1909,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,72.0
28,c0d210398dee4a0895b24444a5fcd1d2,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,168.0
30,57dd18ec5ddc46828afb81ec5977bef2,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,168.0


In [8]:
offers_made.head()

,offer_id,time,count,duration,reward,end
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1297,10,5,240
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,168,1245,10,5,408
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,336,1322,10,5,576
3,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,1269,10,5,648
4,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1221,7,3,168


In [9]:
offers_completed=transcript[transcript.event=='offer completed'].copy()
offers_completed['offer_id']=offers_completed.value.apply(lambda x: x['offer_id'])
offers_completed.drop(['value','event'],axis=1, inplace=True)
offers_completed=offers_completed.merge(portfolio.reward, left_on='offer_id', right_index=True)

offers_completed.head()

,person,time,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5,2
12764,73ffefd41e9a4ca3ab26b2b3697c6eb7,0,2906b810c7d4411798c6938adc9daaa5,2
12786,24115a61df25473e84a8a03f3c98de1a,0,2906b810c7d4411798c6938adc9daaa5,2
12826,2481f1fcfbcb4b288e5a03af02d95373,0,2906b810c7d4411798c6938adc9daaa5,2
13183,a45b69f1c8554ae7af83e74426ca437a,0,2906b810c7d4411798c6938adc9daaa5,2


In [10]:
transactions=transcript[transcript.event=='transaction'].copy()
transactions.value=transactions.value.apply(lambda x: x['amount'])
transactions.set_index('person', inplace=True)
transactions.head()

,event,value,time
person,,,
02c083884c7d45b39cc68e1314fec56c,transaction,0.83,0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,34.56,0
54890f68699049c2a04d415abc25e717,transaction,13.23,0
b2f1cd155b864803ad8334cdf13c4bd2,transaction,19.51,0
fe97aa22dd3e48c8b143116a8403dd52,transaction,18.97,0


In [11]:
def pull_relevant(offer,start,end):
    is_not_null=offers_recieved.set_index('person').apply(
        lambda row: start <= row.time <= end or start <= row.end <= end, axis=1).groupby('person').any()
    not_null=profile.loc[is_not_null.index][is_not_null]
    null=profile.drop(not_null.index)
    offered=offers_recieved.set_index(['time','offer_id']).sort_index().loc[(start,offer),'person'].values
    offered=not_null.loc[offered]
    
    period_transactions=transactions[(start <= transactions.time) & (transactions.time <= end)].value
    
    merge_group = lambda L,R: L.merge(R, left_index=True, right_index=True, how='left').groupby(level=0)
    collate =  lambda X: pd.DataFrame([X.count(),X.sum(),X.mean()], index=['count', 'total', 'mean']).transpose().fillna(0)
    
    null_transactions=merge_group(null,period_transactions).value
    null=null.merge(collate(null_transactions), left_index=True, right_index=True)
    #null.join(null,collate(null_transactions))
    
    offered_transactions=merge_group(offered,period_transactions).value
    offered=offered.merge(collate(offered_transactions), left_index=True, right_index=True)
    
    return null,offered
    
                        

In [12]:
from scipy.stats import ttest_ind

In [13]:
offer, start, end = '0b1e1539f2cc45b7b9fa7c272da2e1d7', 0, 240

In [14]:
def t_test(offer,start,end, cost):
    null,offered=pull_relevant(offer,start,end)
    cols=['age', 'income']
    p_dict={('p_'+col):ttest_ind(null[col].dropna(), offered[col].dropna()).pvalue for col in cols}
    
    
    def t_test_1_sided(a,b,c):
        
        p=ttest_ind(a.dropna()-c, b.dropna(), equal_var=False).pvalue/2
        if a.mean()-c < b.mean():
            p=1-p
        return -np.log(p)
    
    p_dict['p_total-cost']=t_test_1_sided(offered['total'], null['total'], cost)
    p_dict['total-cost']=offered['total'].mean()-null['total'].mean()-cost
    
    for col in ['total','mean', 'count']:
        p_dict['p_'+col]=t_test_1_sided(offered[col],null[col],0)
        p_dict['offered_'+col]=offered[col].mean()
        p_dict['null_'+col]=null[col].mean()
        
    
    p_series=pd.Series(p_dict)
    
    return p_series 

In [15]:
t_test(offer,start,end, 5/2)

p_age             0.637936
p_income          0.151557
p_total-cost     17.774222
total-cost       11.386614
p_total          25.257765
offered_total    29.393778
null_total       15.507164
p_mean            6.989201
offered_mean     11.080959
null_mean         8.311168
p_count          80.160743
offered_count     2.087124
null_count        1.268877
dtype: float64

In [21]:
offers_made.offer_id.unique()

array(['0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       '2906b810c7d4411798c6938adc9daaa5',
       '3f207df678b143eea3cee63160fa8bed',
       '4d5c57ea9a6940dd891ad53e9dbe8da0',
       '5a8bc65990b245e5a138643cd4eb9837',
       '9b98b8c7a33c4b65b9aebfe6a799e6d9',
       'ae264e3637204a6fb9bb56bc8210ddfd',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       'fafdcd668e3743c1bb461111dcafc2a4'], dtype=object)

In [25]:
offers_made.set_index('offer_id', inplace=True)

In [28]:
def t_test(offer,start,end, cost):
    null,offered=pull_relevant(offer,start,end)
    cols=['age', 'income']
    p_dict={('p_'+col):ttest_ind(null[col].dropna(), offered[col].dropna()).pvalue for col in cols}
    return pd.Series(p_dict)

In [44]:
for x in offers_made.reset_index():
    print(x)

offer_id
time
count
duration
reward
end


In [81]:
all_null=[]
all_offered=[]
for i in offers_made.reset_index().index:
    row = offers_made.iloc[i]
    null, offered =pull_relevant(row.name,row.time,row.end)
    null['offer']=row.name
    offered['offer']=row.name
    all_null.append(null)
    all_offered.append(offered)
    
null=pd.concat(all_null).set_index('offer')
offered=pd.concat(all_offered).set_index('offer')

In [82]:
def t_test(null, offered, cost):
    cols=['age', 'income']
    p_dict={('p_'+col):ttest_ind(null[col].dropna(), offered[col].dropna()).pvalue for col in cols}
    
    
    def t_test_1_sided(a,b,c):
        
        p=ttest_ind(a.dropna()-c, b.dropna(), equal_var=False).pvalue/2
        if a.mean()-c < b.mean():
            p=1-p
        return -np.log(p)
    
    p_dict['p_total-cost']=t_test_1_sided(offered['total'], null['total'], cost)
    p_dict['total-cost']=offered['total'].mean()-null['total'].mean()-cost
    
    for col in ['total','mean', 'count']:
        p_dict['p_'+col]=t_test_1_sided(offered[col],null[col],0)
        p_dict['offered_'+col]=offered[col].mean()
        p_dict['null_'+col]=null[col].mean()
        
    
    p_series=pd.Series(p_dict)
    
    return p_series 

In [83]:
p_dict={}
for offer in portfolio.index:
    p_dict[offer]=t_test(null.loc[offer], 
            offered.loc[offer],
            portfolio.loc[offer,'reward'])

<ipython-input-82-a9581915f93b>:11: RuntimeWarning: divide by zero encountered in log
  return -np.log(p)


In [84]:
res_df=pd.DataFrame(p_dict).transpose()

In [89]:
res_df

,p_age,p_income,p_total-cost,total-cost,p_total,offered_total,null_total,p_mean,offered_mean,null_mean,p_count,offered_count,null_count
ae264e3637204a6fb9bb56bc8210ddfd,0.973763,0.444816,17.898564,4.459426,161.321675,27.030298,12.570872,20.848200,10.543121,8.145991,inf,2.275419,0.995231
4d5c57ea9a6940dd891ad53e9dbe8da0,0.785853,0.900341,8.264400,2.523007,149.123879,22.459247,9.936240,44.365033,10.048009,6.900405,inf,1.823623,0.793412
3f207df678b143eea3cee63160fa8bed,0.953140,0.490974,34.590683,3.691419,34.590683,12.517257,8.825838,11.202795,7.376885,6.324209,143.870097,1.004712,0.703473
9b98b8c7a33c4b65b9aebfe6a799e6d9,0.352037,0.609647,12.019896,3.128718,67.632159,20.699589,12.570872,7.209645,9.325117,8.145991,290.008075,1.634971,0.995231
0b1e1539f2cc45b7b9fa7c272da2e1d7,0.839873,0.513380,36.834556,6.974734,102.447217,25.553793,13.579059,16.186718,10.780745,8.406753,366.870206,1.861485,1.077408
2298d6c36e964ae4a3e7e9706d1fb8c2,0.577051,0.843108,152.085582,13.001292,226.744437,28.572164,12.570872,24.204939,10.582903,8.145991,inf,2.341260,0.995231
fafdcd668e3743c1bb461111dcafc2a4,0.984573,0.801376,252.497165,24.977783,292.291603,40.556843,13.579059,34.155231,11.964087,8.406753,inf,3.149215,1.077408
5a8bc65990b245e5a138643cd4eb9837,0.945320,0.678061,81.606298,5.353804,81.606298,12.474020,7.120216,31.613925,7.510596,5.353965,398.933821,1.080629,0.561690
f19421c1d4aa40978ebb69ca19b0e20d,0.886190,0.449848,58.427053,7.873722,149.083129,22.809962,9.936240,41.376852,10.612626,6.900405,inf,1.764682,0.793412
2906b810c7d4411798c6938adc9daaa5,0.793817,0.544438,47.929749,7.173675,76.370747,21.744546,12.570872,13.584005,9.943264,8.145991,298.536525,1.640000,0.995231


The table above shows $-log(p)$ for various $p$-tests. A score above 3 can be considered significant. The key figures here are that age and income seem similarly distributed across null and test groups; and that the $p_{total-cost}$ value is always large. This means that we can reject the hypthesis that the total spend of people to whom the offer was made was lower than the total spend of those who it wasn't plus the monetary value of the offer itself. That is the impact of the offer was both significant and meaningful. Although these tests were aggregated into one number for each offer, the information used was only from the periods in which the offer was active. 

Notcieably there was an increase in both mean spend and number of transactions among offered groups. 

In [93]:
pd.concat([(res_df['total-cost']/portfolio.duration).sort_values(ascending=False), portfolio], axis=1)

,0,reward,difficulty,duration,offer_type,email,mobile,social,web
fafdcd668e3743c1bb461111dcafc2a4,2.497778,2,10,10,discount,1.0,1.0,1.0,1.0
2298d6c36e964ae4a3e7e9706d1fb8c2,1.857327,3,7,7,discount,1.0,1.0,1.0,1.0
5a8bc65990b245e5a138643cd4eb9837,1.784601,0,0,3,informational,1.0,1.0,1.0,0.0
f19421c1d4aa40978ebb69ca19b0e20d,1.574744,5,5,5,bogo,1.0,1.0,1.0,1.0
2906b810c7d4411798c6938adc9daaa5,1.024811,2,10,7,discount,1.0,1.0,0.0,1.0
3f207df678b143eea3cee63160fa8bed,0.922855,0,0,4,informational,1.0,1.0,0.0,1.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,0.697473,5,20,10,discount,1.0,0.0,0.0,1.0
ae264e3637204a6fb9bb56bc8210ddfd,0.637061,10,10,7,bogo,1.0,1.0,1.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,0.504601,10,10,5,bogo,1.0,1.0,1.0,1.0
9b98b8c7a33c4b65b9aebfe6a799e6d9,0.446960,5,5,7,bogo,1.0,1.0,0.0,1.0


While all the measures are deemed effective by our p_tests, there does seem to be differences in performance. Roughly speaking it looks like the 2,10,10 discount performs best, but comparing the offers like this is on shaky statistical footing. There may be other factors impacting performance, e.g. people might be more likely to buy coffee on a monday, so offers which include a (or multiple) mondays might get a boost. 